# Playing with Sound (.wav files) with OpenCV, numpy and matplotlib

#### Also the introduction to the use of IPython Display and ipywidgets

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 8]

In [ ]:
from scipy.io import wavfile as wf

In [ ]:
import soundfile as sf # to convert the 24 bit-depth wav file to 16 or 32 bit-depth wav file but not necessary here

In [ ]:
sample_rate, data = wf.read('flute.wav')
plt.plot(data);

In [ ]:
import IPython.display as ipd
ushort_max = len(data)
print(ushort_max)
ipd.Audio(data = data, rate = sample_rate)

### More on IPython Display-
Class Representation is normally done using the `__repr__()` method

In [ ]:
x = [1,2,3,4]
x.__repr__()

In [ ]:
x

But in Jupyter there is the `_repr_html_()` method. We can apply it a user defined custom class as well.

In [ ]:
# THe following code gives a customised html output to a user defined list using the concept of classes and objects.

class MyList(object):
    
    def __init__(self, items):
        self.items = items
        
    def _repr_html_(self):
        out = "<ul style = 'background: #e0e0ff; margin: 2em; border: 1px solid gray; padding: 25px'>"
        for i, v in enumerate(self.items):
            out += "<li><i> Item {}:</i> {}</li>".format(i, str(v))
        out += "</ul>"
        return out

In [ ]:
x = MyList(["Apple", "Apple", "Samsung", "Orange", "OnePlus"])
x 

In [ ]:
squeakydata = data[::2]
ipd.Audio(data = squeakydata, rate = sample_rate)

In [ ]:
slowdata = np.interp(
    x = np.arange(0, len(data), 0.67),
    xp = np.arange(len(data)),
    fp = data
)
ipd.Audio(data = slowdata, rate = sample_rate)

### Making a Chorus

In [ ]:
import random

def make_chorus(data, chorus_size, max_offset, speed_min, speed_max):
    chorus = np.zeros(int(len(data) / speed_min) + max_offset)
    for i in range(chorus_size):
        offset = random.randint(0, max_offset)
        speed = random.uniform(speed_min, speed_max)
        new_length = int(len(data) / speed)
        sample = np.interp(
        x  = np.linspace(0, len(data), num = new_length),
        xp = np.arange(len(data)),
        fp = data
        )
        chorus[offset: (offset+new_length)] += sample
                        
    return chorus

chorus = make_chorus(data, chorus_size = 4, max_offset = 7000, speed_min = 0.9, speed_max = 1.1)

ipd.Audio(data = chorus, rate = sample_rate)

### Interactive interface in Jupyter Notebooks.

We can tweak the parameters to our liking using the tool ipywidgets library in iPython.

In [ ]:
import ipywidgets as widgets
w = widgets.IntSlider(min=0, max=20)
w # An example of using an interactive slider in iPython

In [ ]:
w.value

In [ ]:
from IPython.display import display

chorus_size = widgets.IntSlider(
    value=4,
    min=0, max=20,
    step=1,
    description='Chorus Size',
    readout=True,
    readout_format='d',
#     style={'description_width':'30%'},
#     layout={'width':'80%'}
)
display(chorus_size)

max_offset = widgets.IntSlider(
    value=4000,
    min=0, max=20000,
    step=1000,
    description='Max. Offset (samples)',
    readout=True,
    readout_format='d',
#     style={'description_width':'30%'},
#     layout={'width':'80%'}
)
display(max_offset)

speed_min = widgets.FloatSlider(
    value=0.9,
    min=0, max=2.0,
    step=0.1,
    description='Min. Speed',
    readout=True,
    readout_format='.1f',
    style={'description_width':'30%'},
    layout={'width':'80%'}
)
display(speed_min)

speed_max = widgets.FloatSlider(
    value=1.2,
    min=0, max=2.0,
    step=0.1,
    description='Min. Speed',
    readout=True,
    readout_format='.1f',
    style={'description_width':'30%'},
    layout={'width':'80%'}
)
display(speed_max)

def new_chorus(change):
    chorus = make_chorus(
    data,
    chorus_size = chorus_size.value,
    max_offset = max_offset.value,
    speed_min = speed_min.value,
    speed_max = speed_max.value
    )
    plt.plot(chorus)
    plt.show()
    display(
        ipd.Audio(
            data = chorus,
            rate = sample_rate
        )
    )
    
go_but = widgets.Button(description = 'Generate', layout={'width': '80%'})
go_but.on_click(new_chorus)
ipd.display(go_but)